# Feature Engineering 2
### Street Level Feature

In [6]:
import pandas as pd
import numpy as np
from plotnine import *
from shapely import wkt
import geopandas as gpd
from datetime import datetime

In [7]:
PATH_RAW = '../data/raw/'
PATH_INTERIM = '../data/interim/'

In [8]:
# 50th Percentile
def q50(x):
    return x.quantile(0.5)
def q75(x):
    return x.quantile(0.75)
def q25(x):
    return x.quantile(0.25)

In [12]:
df = pd.read_csv(PATH_INTERIM+"iterim1.csv").iloc[:,2:]

In [14]:
df.sample(2)

,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,level,median_length,median_delay,median_speed_kmh,total_records,id,...,total_records_holiday_sum,total_records_holiday_q25,total_records_holiday_q75,total_records_holiday_mean,total_records_holiday_q50,total_records_busy_hour_mean,total_records_busy_hour_sum,total_records_busy_hour_q25,total_records_busy_hour_q50,total_records_busy_hour_q75
163762,2022-08-08 15:00:00,32.73,KOTA BANDUNG,Ir Haji Juanda,2,1335.0,125.0,16.689999,84,35608346,...,2012056,4.0,25.0,21.431526,10.0,54.976169,304513.0,4.0,10.0,118.0
209857,2022-08-17 18:00:00,32.73,KOTA BANDUNG,Akses Tol Pasteur,3,187.0,90.5,4.780000,8,36146780,...,4433653,4.0,25.0,21.304168,11.0,54.976169,304513.0,4.0,10.0,118.0


In [21]:
# street is repeated, but it combined with different jams level
print(len(df.street.unique()))

1162


#### Street + weekend and Street + holiday

In [44]:
street_weekend = []
for i, rows in df.iterrows():
  street_weekend.append(str(rows['street']) + "_" +str(rows['is_weekend']))

In [46]:
df['street_weekend'] = street_weekend

In [50]:
street_holiday = []
for i, rows in df.iterrows():
  if(rows['is_holiday'] and not rows['is_weekend']):
    street_holiday.append(str(rows['street']) + "_" +str(rows['is_holiday']))
  else:
    street_holiday.append(str(rows['street']) + "_0")

In [51]:
df['street_holiday'] = street_holiday

In [52]:
df.sample(2)

,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,level,median_length,median_delay,median_speed_kmh,total_records,id,...,total_records_holiday_mean,total_records_holiday_q50,total_records_busy_hour_mean,total_records_busy_hour_sum,total_records_busy_hour_q25,total_records_busy_hour_q50,total_records_busy_hour_q75,street_level,street_weekend,street_holiday
192836,2022-08-14 09:00:00,32.73,KOTA BANDUNG,Otto Iskandardinata,3,728.0,130.0,12.48,36,35974099,...,21.304168,11.0,54.976169,304513.0,4.0,10.0,118.0,Otto Iskandardinata_3,Otto Iskandardinata_1,Otto Iskandardinata_0
268464,2022-08-29 13:00:00,32.73,KOTA BANDUNG,Jalan Malabar,3,239.0,107.0,5.54,13,36850223,...,21.431526,10.0,54.976169,304513.0,4.0,10.0,118.0,Jalan Malabar_3,Jalan Malabar_0,Jalan Malabar_0


#### Metric on street_holiday and street_weekend

1. total_records:
avg, q25, q50, q75
2. level:
avg, q25, q50, q75

street_weekend

In [62]:
grp_street_weekend = df.groupby('street_weekend')

In [65]:
metrics = [np.mean, q25, q50, q75]
metrics_name = ["avg", "q25", "q50", "q75"]
col_func = {}
col_func_name = ['street_weekend']

for col in ['total_records', 'level']:
  col_func[col] = metrics
  for metric in metrics_name:
    col_func_name.append(col+"_"+metric)  

In [78]:
agg_street_weekend = grp_street_weekend.agg(col_func).reset_index()
agg_street_weekend.columns = col_func_name
# drop level_avg (it's categorical data)
agg_street_weekend = agg_street_weekend.drop(columns=['level_avg'])

In [79]:
agg_street_weekend.sample(4)

,street_weekend,total_records_avg,total_records_q25,total_records_q50,total_records_q75,level_q25,level_q50,level_q75
1725,Taman Cibaduyut Indah_1,3.428571,1.0,2.0,5.0,2.0,3.0,4.0
282,Cigending_0,7.000000,3.0,7.0,10.0,1.0,2.0,2.0
981,Kuta Kencana Timur_0,1.000000,1.0,1.0,1.0,4.0,4.0,4.0
8,Adi Flora Raya_1,7.333333,4.0,9.0,9.0,1.0,2.0,2.0


street_holiday

In [82]:
grp_street_holiday = df.groupby('street_holiday')

In [84]:
metrics = [np.mean, q25, q50, q75]
metrics_name = ["avg", "q25", "q50", "q75"]
col_func_h = {}
col_func_name_h = ['street_holiday']

for col in ['total_records', 'level']:
  col_func_h[col] = metrics
  for metric in metrics_name:
    col_func_name_h.append(col+"_"+metric)

In [85]:
agg_street_holiday = grp_street_holiday.agg(col_func_h).reset_index()
agg_street_holiday.columns = col_func_name_h
# drop level_avg (it's categorical data)
agg_street_holiday = agg_street_holiday.drop(columns=['level_avg'])

In [86]:
agg_street_holiday.sample(5)

,street_holiday,total_records_avg,total_records_q25,total_records_q50,total_records_q75,level_q25,level_q50,level_q75
563,Ir Haji Juanda_0,48.966548,10.0,33.0,76.0,2.0,2.0,3.0
379,Derwati_0,9.787018,4.0,9.0,13.0,1.0,2.0,2.0
448,Gang Haji Mesri_1,4.800000,3.0,4.0,7.0,2.0,2.0,3.0
499,Golf Raya_0,6.515464,2.0,6.0,10.0,2.0,2.0,3.0
1006,N11 Jenderal Sudirman_0,12.237958,4.0,9.0,17.0,2.0,3.0,3.0


### Merge

In [ ]:
pd.concat([df, ], axis=1, join="inner")